# Intro

Welcome to the interactive labeling notebook!

The notebook proceeds as follows:

1. **Import** libraries
2. Define **paths** to data
3. **Label** ROI images using a simple GUI
4. **Visualize** results
5. **Save** results


- If you need to restart a session, don't worry. Your results are continuously saved in the CSV file and you can just start from where you left off.
- We recommend labeling one session at a time so that the results can be saved with the data.

##### If running on google colab:

- install roicat

After running the cell below, the runtime will restart.

In [16]:
#@title Install `roicat` if on colab
using_colab = 'google.colab' in str(get_ipython())

if using_colab:
    !pip uninstall -y tensorflow
    !pip install roicat[classification]

In [17]:
#@title mount gdrive if on colab
#@markdown Upload your data to Google Drive, then mount the drive and access the cloud directory here.
#@markdown You can use the sidebar to the left to browse your google drive directories.

using_colab = 'google.colab' in str(get_ipython())

if using_colab:
    from google.colab import drive
    path_gdrive = '/content/gdrive'
    drive.mount(path_gdrive, force_remount=True)

In [ ]:
if using_colab:
    from google.colab import output
    output.enable_custom_widget_manager()

# Import libraries

Widen the notebook

In [ ]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

Import basic libraries

In [2]:
from pathlib import Path
import tempfile

import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt

Import `roicat`

In [3]:
import roicat

# Find paths to data

##### Prepare list of paths to data

In this example we are using suite2p output files, but other data types can be used (CaImAn, etc.) \
See the notebook on ingesting diverse data: https://github.com/RichieHakim/ROICaT/blob/main/notebooks/jupyter/other/demo_data_importing.ipynb

Make a list containing the paths to all the input files.

In this example we are using suite2p, so the following are defined:
1. `paths_allStat`: a list to all the stat.npy files
2. `paths_allOps`: a list with ops.npy files that correspond 1-to-1 with the stat.npy files

In [ ]:
dir_data = r'/media/rich/bigSSD/other_lab_data/Allen_institute/David_Feng'

paths_stat = roicat.helpers.find_paths(
    dir_outer=dir_data,
    reMatch='stat\.npy',
    depth=5,
)

display(paths_stat)

paths_ops = [str(Path(p).parent / 'ops.npy') for p in paths_stat]

display(paths_ops)

In [ ]:
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_stat,
    paths_opsFiles=paths_ops,
    new_or_old_suite2p='new',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

# ROInet embedding

This step passes the images of each ROI through the ROInet neural network. The inputs are the images, the output is an array describing the visual properties of each ROI.

##### 1. Initialize ROInet

Initialize the ROInet object. The `ROInet_embedder` class will automatically download and load a pretrained ROInet model. If you have a GPU, this step will be much faster.

In [ ]:
DEVICE = roicat.helpers.set_device(use_GPU=True, verbose=True)
dir_temp = tempfile.gettempdir()

roinet = roicat.ROInet.ROInet_embedder(
    device=DEVICE,  ## Which torch device to use ('cpu', 'cuda', etc.)
    dir_networkFiles=dir_temp,  ## Directory to download the pretrained network to
    download_method='check_local_first',  ## Check to see if a model has already been downloaded to the location (will skip if hash matches)
    download_url='https://osf.io/c8m3b/download',  ## URL of the model
    download_hash='357a8d9b630ec79f3e015d0056a4c2d5',  ## Hash of the model file
    forward_pass_version='head',  ## How the data is passed through the network
    verbose=True,  ## Whether to print updates
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,  ## Input images of ROIs
    um_per_pixel=data.um_per_pixel,  ## Resolution of FOV
    pref_plot=False,  ## Whether or not to plot the ROI sizes
);
images = roinet.ROI_images_rs

##### 2. Check ROI_images sizes
In general, you want to see that a neuron fills roughly 25-50% of the area of the image. \
**Adjust `um_per_pixel` above to rescale image size**

In [ ]:
roicat.visualization.display_toggle_image_stack(images[:1000], image_size=(200,200))

##### 3. Pass data through network

Pass the data through the network. Expect for large datasets (~40,000 ROIs) that this takes around 15 minutes on CPU or 1 minute on GPU.

In [ ]:
roinet.generate_latents();

# UMAP embedding

Reduce the dimensionality of the output of ROInet (~100 dims) to 2 dimensions so that we can visualize it. Feel free to use any settings here that do a good job of clustering your data as you see fit.

In [ ]:
model_umap = UMAP(
    n_neighbors=25,
    n_components=2,
    n_epochs=400,
    verbose=True,
    densmap=False,
)
emb = model_umap.fit_transform(roinet.latents)

# Label data

How to use this GUI:
- Each image will be shown one at a time. You can go to previous and next images by using the ***Left*** and ***Right*** arrow keys, and can close the GUI by pressing the ***Escape*** key.
- An embedding of the ROIs will be shown on the right with example images overlaid onto the scatterplot. You can use the lasso tool to draw a region around a section of the scatter plot to select ROIs for labeling, the selected ROIs will be made available to move through and label on the left. 
- You can turn off the overlays by pressing Control-Shift-T

Labeling:
- Identify several target regions in the embedding space. Lasso each region, and label a bunch of examples from within that region. 
- Label the image by pressing any key on your keyboard. This key will be used as the label. After each key press, the following occurs:
    1. The results will be saved as a CSV file at `path_csv` (will be a temporary file if not specified)
    2. The results will also be stored in the python object as `ImageLabeler_obj.labels_`.
    2. The GUI should progress to the next image.
    3. Finish labeling by closing out of the GUI window or pressing the user defined '`end_key`' (***Escape*** by default).
- The results are stored in a temporary CSV file with two columns: 'index' and 'label'. This file can be used to recover lost labeling session data. We will save more comprehensive results files at the end of the notebook.
    - **'index'**: The index within the data object. 0-indexed.
    - **'label'**: The key pressed for that image in the labeling GUI.

In [11]:
# The images are what we will label and will be shown on the left
images = np.concatenate(data.ROI_images, axis=0)

# The idx to the overlay images is designed to pick images to show that
# are well spread out in the embedding space from UMAP
idx_images_overlay = roicat.visualization.get_spread_out_points(
    emb,
    n_ims=min(emb.shape[0], 1500),  ## Select number of overlayed images here
    dist_im_to_point=0.8,
)

In [ ]:
with roicat.helpers.IntegratedLabeler(
    images,
    embeddings=emb,
    idx_images_overlay=idx_images_overlay,
    size_images_overlay=0.25,
    frac_overlap_allowed=0.25,
    crop_images_overlay=0.25,
    alpha_points=1.0,
    size_points=3.0,
) as labeler:
    labeler.run()
    
labels = {
    'labels_list': labeler.get_labels(kind='list'),
    'labels_dict': labeler.get_labels(kind='dict'),
    'labels_df': labeler.get_labels(kind='dataframe'),
}

print(f"Number of labeled ROIs: {len(labels['labels_list'])}")
print(f"path to csv file: {labeler.path_csv}")

In [13]:
labels = {
    'list': roicat.util.JSON_List(labeler.get_labels(kind='list')),
    'df': labeler.get_labels(kind='dataframe'),
}

# Visualize results

In [ ]:
u, c = np.unique(labels['df']['label'], return_counts=True)

plt.figure()
plt.bar(u, c);
plt.xlabel('label');
plt.ylabel('counts');

In [ ]:
roicat.visualization.display_labeled_ROIs(
    images=images,
#     labels=labels['label'].astype(np.int64),
    labels=labels['df'],
    max_images_per_label=10,
    figsize=(10, 3),
    fontsize=25,
    shuffle=False,
)

# Save results

The results file can be opened using any of the following methods:
1. `roicat.helpers.pickle_load(path)`
2. `np.load(path)`
3. 
``` 
import pickle
with open(path_save, mode='rb') as f:
    test = pickle.load(f)
  ```

##### Make and save results

In [109]:
run_data = {
    "data": data.__dict__,
}
results = {
    "labels": labels,
    "images": images,
}

In [ ]:
## Define the directory to save the results to
dir_save = '/media/rich/bigSSD/data_tmp/test_data/'
name_save = 'mouse_1'

paths_save = {
    'labels':   str(Path(dir_save) / f'{name_save}.labeling.labels.csv'),
    'results':  str(Path(dir_save) / f'{name_save}.labeling.results.richfile'),
    'run_data': str(Path(dir_save) / f'{name_save}.labeling.run_data.richfile'),
}

## Save
labels['df'].to_csv(paths_save['labels'])
roicat.util.RichFile_ROICaT(path=paths_save['results']).save(results, overwrite=True)
roicat.util.RichFile_ROICaT(path=paths_save['run_data']).save(run_data, overwrite=True)

# Thank you
If you encountered any difficulties, please let us know at the issues page: https://github.com/RichieHakim/ROICaT/issues

---
# Extras

## Merge and convert GUI CSV files into a results file

##### 1. Make labels dictionary

In [99]:
import csv

# path_csv = r'/home/rich/Desktop/test.csv'  ## Path to the raw output .csv file from the GUI
path_csv = labeler.path_csv

with open(path_csv, 'r') as f:
    lines = np.array([line for line in csv.reader(f)], dtype=str)

labels_dict = {int(idx): label for idx, label in zip(lines[1:, 0], lines[1:, 1])}

##### 2. Make images array

In [ ]:
dir_data = r'/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_0322R/statFiles/20230420/'


path_stat = str(Path(dir_data) / 'stat.npy')
path_ops = str(Path(dir_data) / 'ops.npy')

data = roicat.data_importing.Data_suite2p(
    paths_statFiles=[path_stat],
    paths_opsFiles=[path_ops],
    new_or_old_suite2p='new',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

images = np.concatenate(data.ROI_images, axis=0)

##### 3. Make the results dictionary

In [103]:
import pandas as pd

labels_list = ['None'] * len(labels_dict)
for idx, label in labels_dict.items():
    labels_list[idx] = label

labels_df = pd.DataFrame(index=list(labels_dict.keys()), data=labels_list, columns=['label'])

labels = {
    'list': roicat.util.JSON_List(labels_list),
    'dict': roicat.util.JSON_Dict(labels_dict),
    'df': labels_df,
}

In [104]:
run_data = {
    "data": data.__dict__,
}
results = {
    "labels": labels,
    "images": images,
}

##### 4. Save the results dictionary

In [ ]:
## Define the directory to save the results to
dir_save = '/media/rich/bigSSD/data_tmp/test_data/'
name_save = 'mouse_1'

paths_save = {
    'labels':   str(Path(dir_save) / f'{name_save}.labeling.labels.json'),
    'results':  str(Path(dir_save) / f'{name_save}.labeling.results.richfile'),
    'run_data': str(Path(dir_save) / f'{name_save}.labeling.run_data.richfile'),
}

roicat.helpers.json_save(obj=roicat.util.JSON_List(labels['list']), filepath=paths_save['labels'])
roicat.util.RichFile_ROICaT(path=paths_save['results']).save(results, overwrite=True)
roicat.util.RichFile_ROICaT(path=paths_save['run_data']).save(run_data, overwrite=True)